In [416]:
# readline_all.py
neg=[]
f = open("neg.txt", 'r')
while True:
    line = f.readline()
    if not line: break
    neg.append(line)
f.close()

# readline_all.py
pos=[]
f = open("pos.txt", 'r')
while True:
    line = f.readline()
    if not line: break
    pos.append(line)
f.close()

In [418]:
def delete_blank(document):
    result=[]
    for i in range(len(document)):
        if i % 2 == 0:
            result.append(document[i])
    return result

pos_Text = delete_blank(pos)
neg_Text = delete_blank(neg)

In [419]:
vailidation_Text = pos_Text+neg_Text

In [420]:
vailidation_result = []
for i in vailidation_Text:
    a = i.split("\n")
    vailidation_result.append(a[0])

In [421]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [422]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [423]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

In [424]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [425]:
import random
random_data = random.sample(review_data,len(review_data))

In [426]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"].lower())

In [428]:
filtered_sentence_result = []
join_text = []
from nltk.tokenize import word_tokenize

for i in reviewText:
    word_tokens = word_tokenize(i)
    filtered_sentence = [w for w in word_tokens if w in vailidation_result]
    filtered_sentence_result.append(filtered_sentence)

In [441]:
join_filtered_sentence_result = []
for i in filtered_sentence_result:
    a = " ".join(i)
    join_filtered_sentence_result.append(a)

In [442]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df=0.3 ,min_df = 2, lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(join_filtered_sentence_result)

In [446]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)

### 비율 맞추기.......?

In [456]:
from sklearn import cross_validation

In [457]:
skf = cross_validation.StratifiedKFold(y_data, n_folds=10,random_state=500)

In [458]:
for train_index, test_index in skf:
    X_train, X_test = D[train_index], D[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

In [459]:
from collections import Counter
Counter(y_test)

Counter({1.0: 4, 2.0: 3, 3.0: 10, 4.0: 22, 5.0: 58})

In [460]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((903, 897), (903,), (97, 897), (97,))

### Multiclass SVM

In [461]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
result.predict(X_test)

array([ 5.,  3.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  4.,  5.,  3.,  4.,
        4.,  5.,  5.,  4.,  5.,  4.,  5.,  5.,  3.,  5.,  5.,  5.,  4.,
        5.,  5.,  3.,  5.,  1.,  3.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        4.,  5.,  5.,  4.,  3.,  5.,  1.,  4.,  5.,  5.,  5.,  5.,  5.,
        5.,  5.,  5.,  5.,  4.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,
        4.,  5.,  5.,  5.,  5.,  4.,  5.,  3.,  4.,  5.,  1.,  5.,  5.,
        5.,  2.,  5.,  5.,  5.,  5.,  4.,  4.,  5.,  5.,  5.,  4.,  5.,
        5.,  4.,  5.,  5.,  5.,  5.])

In [462]:
from sklearn.metrics import accuracy_score
accuracy_score(result.predict(X_test),y_test)

0.53608247422680411

### LogisticRegression

In [463]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(X_train, y_train)
logreg.predict(X_test) == y_test

array([False, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False, False,
       False,  True, False,  True, False,  True,  True, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True, False,
       False,  True, False, False, False,  True, False,  True, False,
       False,  True, False, False,  True,  True,  True,  True,  True,
       False,  True, False, False, False, False,  True, False,  True,
       False,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True, False, False,
        True, False,  True,  True, False, False, False,  True, False,
        True, False, False, False,  True, False,  True], dtype=bool)

In [464]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(X_test), y_test)

0.58762886597938147